$$
\newcommand{\Real}{\mathbb R}
\newcommand{\Complex}{\mathbb C}
\newcommand{\F}{\mathcal F}
$$

# Pseudospectral solution of the incompressible Navier-Stokes equations

In this notebook, we'll solve the equations describing flow of an incompressible, viscous fluid in 2D using a pseudospectral discretization.

## The equations

We will present the equations for an arbitrary number of dimensions $d \in \{1,2,3\}$.

### Conservation laws
We start with the familiar continuity equation for mass conservation:

$$\rho_t + \nabla \cdot (\rho u) = 0.$$

Here $\rho$ is mass density and $u(x,t) \in \Real^d$ is fluid velocity.  This equation states that the change in mass in a region is due to flow of mass (whose rate is $\rho u$) into or out of that region.

Similarly, we have the following equation for the flow of momentum:

$$(\rho u)_t + \nabla \cdot \left( \rho u \otimes u \right) + \nabla p = \nu \nabla^2 u.$$

Here $p$ is the fluid pressure, $\nu$ is the coefficient of viscosity, and $\otimes$ denotes the outer product.  The term $\nabla \cdot \left( \rho u \otimes u \right)$ is similar to the term $\nabla \cdot (\rho u)$ in the continuity equation -- it accounts for transport of momentum due to motion of the fluid.  Meanwhile the term $\nabla P$ accounts for momentum exchange due to pressure differences, and the right-hand-side term accounts for viscosity.

### Incompressibility

Let us now suppose that the fluid is incompressible.  This means that the fluid density does not change in time.  We will go one step further and assume that the density is initially uniform in space, so that it remains constant always.  For simplicity we choose units so that $\rho=1$.  Then the continuity equation simplifies; since $\rho_t = 0$ and

$$\nabla \cdot (\rho u) = u \cdot \nabla \rho + \rho (\nabla \cdot u) = \nabla \cdot u$$

the continuity equation is now simply

$$\nabla \cdot u = 0.$$

The momentum equation also simplifies; we immediately obtain

$$u_t + u \cdot \nabla u + \nabla p = \nu \nabla^2 u.$$

Next we use the identity 

$$u \cdot \nabla u = \frac{1}{2} \nabla (u \cdot u) - u \times (\nabla \times u).$$

Defining the *modified pressure* $P = p + \frac{1}{2}u\cdot u$ and the *vorticity* $\omega = \nabla \times u$, we obtain the "rotational form" of the momentum equation:

$$u_t - u\times \omega = \nu \nabla^2 u - \nabla P.$$

# Pseudospectral discretization

We are thus left with the $d$ evolution equations

$$u_t - u\times \omega = \nu \nabla^2 u - \nabla P,$$

and the constraint

$$\nabla \cdot u = 0.$$

Taking the Fourier transform of these we obtain an ODE and an algebraic constraint for each wavenumber vector $\xi \in \Complex^d$:

\begin{align}
    \hat{u}'_\xi(t) & = (\widehat{u\times \omega})_\xi -\nu |\xi|^2 \hat{u}_\xi - i \xi \hat{P}_\xi \\
    i \xi \cdot \hat{u}_\xi & = 0.
\end{align}

We can take the dot product of the first equation with $i \xi$ and use the second equation to obtain
$$ \hat{P}_\xi = - i \frac{\xi \cdot(\widehat{u\times \omega})_\xi}{|\xi|^2}.$$

Substituting this back into the ODEs gives

$$ \hat{u}'_\xi(t) = (\widehat{u\times \omega})_\xi -\nu |\xi|^2 \hat{u}_\xi - \xi \frac{\xi \cdot(\widehat{u\times \omega})_\xi}{|\xi|^2}.$$

The (linear) viscous term can be handled spectrally, but for the other two terms we will need the pseudospectral approach: compute $\omega$ in Fourier space, transform $u, \omega$ to physical space, take the cross product, and transform the result back to Fourier space.

## Multi-dimensional Fourier transforms

In 2D, the discrete Fourier transform looks like
$$\F_\xi(u) = \F_{\xi_x} ( \F_{\xi_y} (u)) = \sum_{j=0}^{N-1} e^{-2\pi i j \xi_x/N} \sum_{k=0}^{N-1} e^{-2\pi i k \xi_y/N} u_\xi$$

where $\xi_x, \xi_y$ each range from $-N/2 + 1$ to $N/2$.

# Implementation

The code below is adapted from Mikael Mortensen's [SpectralDNS code](https://github.com/mikaem/spectralDNS).  It has been simplified in several ways.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from numpy import exp, pi, cos, sin, cosh
from numpy import fft
import matplotlib
from matplotlib import animation
from JSAnimation import IPython_display # if you don't have this, get it from: https://github.com/jakevdp/JSAnimation

In [ ]:
def wavenumbers(N):
    Nf = N/2+1
    # Set wavenumbers in grid
    kx = fft.fftfreq(N, 1./N)
    ky = kx[:Nf].copy(); ky[-1] *= -1
    K = np.array(np.meshgrid(kx, ky[:Nf], indexing='ij'), dtype=int)
    K2 = np.sum(K*K, 0)
    K_over_K2 = np.array(K, dtype=float) / np.where(K2==0, 1, K2)

    return K, K2, K_over_K2

In [ ]:
def initial_data(problem,N,L):
    X = np.mgrid[:N, :N].astype(float)*L/N

    K, K2, K_over_K2 = wavenumbers(N)
    Nf = N/2+1
    U     = np.empty((2, N, N))
    U_hat = np.empty((2, N, Nf), dtype="complex")
    if problem == 'Taylor-Green':
        U[0] = sin(X[0])*cos(X[1])
        U[1] =-cos(X[0])*sin(X[1])
    elif problem == 'vortices':
        w =     exp(-((X[0]-pi)**2+(X[1]-pi+pi/4)**2)/(0.2)) \
           +    exp(-((X[0]-pi)**2+(X[1]-pi-pi/4)**2)/(0.2)) \
           -0.5*exp(-((X[0]-pi-pi/4)**2+(X[1]-pi-pi/4)**2)/(0.4))
        w_hat = U_hat[0].copy()
        w_hat = fft.rfft2(w)
        U[0] = fft.irfft2( 1j*K_over_K2[1]*w_hat)
        U[1] = fft.irfft2(-1j*K_over_K2[0]*w_hat)
    elif problem =='double-shear':
        delta = 1./20
        sigma = 15/pi
        x, y = X[0], X[1]
        w = (delta * cos(x) - sigma/(cosh(sigma*(y-pi/2)))**2) * (y<=pi)
        w +=(delta * cos(x) + sigma/(cosh(sigma*(3*pi/2-y)))**2) * (y>pi)
        w_hat = U_hat[0].copy()
        w_hat = fft.rfft2(w)
        U[0] = fft.irfft2( 1j*K_over_K2[1]*w_hat)
        U[1] = fft.irfft2(-1j*K_over_K2[0]*w_hat)
        
    # Fourier transform initial data
    U_hat[0] = fft.rfft2(U[0])
    U_hat[1] = fft.rfft2(U[1])

    # Make it divergence free in case it is not
    U_hat[:] -= np.sum(K*U_hat, 0)*K_over_K2    

    return U, U_hat

In [ ]:
def solve(M=6, temporal='RK4', plot_result=10, nu=1.e-3, dt=5.e-2, 
          T=10., problem='vortices', debug=False, errtol=1.e-5,
          make_plots=True):
    # Set the size of the doubly periodic box N**2
    N = 2**M
    L = 2 * pi
    X = np.mgrid[:N, :N].astype(float)*L/N

    # Solution array and Fourier coefficients
    # Because of real transforms and symmetries, N/2+1 coefficients are sufficient
    Nf = N/2+1

    P_hat = np.empty((N, Nf), dtype="complex")
    curl   = np.empty((N, N))

    U_hat0 = np.empty((2, N, Nf), dtype="complex")
    dU     = np.empty((2, N, Nf), dtype="complex")

    U_hat1 = np.empty((2, N, Nf), dtype="complex")
    b = np.array([1./6., 1./3., 1./3., 1./6.])
    c = np.array([0.5, 0.5, 1.])

    def ComputeRHS(U_hat, dU):
        U[0] = fft.irfft2(U_hat[0])
        U[1] = fft.irfft2(U_hat[1])

        curl[:] = fft.irfft2(1j*(K[0]*U_hat[1] - K[1]*U_hat[0]))
        dU[0] = fft.rfft2(U[1]*curl)
        dU[1] = fft.rfft2(-U[0]*curl)

        # Compute pressure (To get actual pressure multiply by 1j/dt)
        P_hat[:] = np.sum(dU*K_over_K2, 0, out=P_hat)

        # Add pressure gradient
        dU[:] -= P_hat*K

        # Add contribution from diffusion
        dU[:] -= nu*K2*U_hat

    K, K2, K_over_K2 = wavenumbers(N)

    U, U_hat = initial_data(problem, N, L)
    curl = fft.irfft2(1j*K[0]*U_hat[1]-1j*K[1]*U_hat[0])

    t = 0.0
    tstep = 0

    C = [curl]
    tt = [t]
    
    while t < T:
        t += dt; tstep += 1

        U_hat1[:] = U_hat0[:] = U_hat
        for j in range(4):
            ComputeRHS(U_hat, dU)
            if j < 3:
                #U_hat[:] = U_hat0 + b[rk]*dU
                U_hat[:] = U_hat0; U_hat += c[j]*dt*dU # Faster (no tmp array)
            U_hat1[:] += b[j]*dt*dU
        U_hat[:] = U_hat1[:]


        if tstep % plot_result == 0 and make_plots:
            curl = fft.irfft2(1j*K[0]*U_hat[1]-1j*K[1]*U_hat[0])
            C.append(curl)
            tt.append(t)

    return tt, C

In [ ]:
tt, C = solve(problem='double-shear', M=7, dt=0.01)

In [ ]:
fig = plt.figure()
axes = fig.add_subplot(111)

im = axes.imshow(np.flipud(C[0].T))
plt.colorbar(im)

def plot_frame(i):
    im.set_data(np.flipud(C[i].T))
    axes.set_title('t= %.2f' % tt[i])
    
matplotlib.animation.FuncAnimation(fig, plot_frame, frames=len(C), interval=20)